## Librerías:

In [8]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import copy
import rasterio

Para los datos Landsat 8 utilizamos las siguientes fórmulas de MNDWI y NDVI:  

*NDWI = (Verde – SWIR)/(Verde + SWIR)*   
**NDWI = (Banda 3 – Banda 6)/(Banda 3 + Banda 6)**  
Los valores del NDWI corresponden a los siguientes rangos:  
0,2 – 1 – Superficie del agua,  
0,0 – 0,2 – Inundación, humedad,  
-0,3 – 0,0 – Sequía moderada, superficies sin agua,  
-1 – -0,3 – Sequía, superficies sin agua  

*NDVI = (Rojo – NIR)/(Rojo + NIR)*  
**NDVI = (Banda 5 - Banda 4)/(Banda 5 + Banda 4)**  


## Carga de imagenes

In [17]:
# Cargamos la imagen y la dividimos por 255 para normalizarla 
path = 'Los-Molinos_2023-05-14_6_all_bands/'
B3= mpimg.imread(path + '2023-05-14.SR_B3.tif') / 255
B4= mpimg.imread(path + '2023-05-14.SR_B4.tif') / 255
B5= mpimg.imread(path + '2023-05-14.SR_B5.tif') / 255
B5= mpimg.imread(path + '2023-05-14.SR_B6.tif') / 255

### Exploración de las imágenes

In [18]:
# Obtenemos las dimensiones
print("Dimensiones de imagen B3:")
[filas, columnas] = B3.shape
print("        Filas = ", filas, "; Columnas = ", columnas)

print("Dimensiones de imagen B4:")
[filas, columnas] = B4.shape
print("        Filas = ", filas, "; Columnas = ", columnas)

print("Dimensiones de imagen B5:")
[filas, columnas] = B5.shape
print("        Filas = ", filas, "; Columnas = ", columnas)

print("Dimensiones de imagen B6:")
[filas, columnas] = B6.shape
print("        Filas = ", filas, "; Columnas = ", columnas)

Dimensiones de imagen B3:
        Filas =  406 ; Columnas =  298
Dimensiones de imagen B4:
        Filas =  406 ; Columnas =  298
Dimensiones de imagen B5:
        Filas =  406 ; Columnas =  298
Dimensiones de imagen B6:


NameError: name 'B6' is not defined